Benchmark results for FindView Task

In [28]:
import os
import sys

import csv
import json

from typing import Any, Dict, List

In [29]:
def get_num_perfects_with_spl(episodes: List[Dict[str, Any]]):
    
    num_perfects = 0
    num_stopped = 0
    spls = []
    
    for i, episode in enumerate(episodes):
        called_stop = episode['called_stop']
        
        spl = 0
        if called_stop:
            num_stopped += 1
            
            l1_distance = episode['l1_distance']
            if l1_distance == 0:
                num_perfects += 1
                
            # if l1_distance < 10:  # success range

                actual = episode['elapsed_steps']
                
                init = episode['initial_rotation']
                target = episode['target_rotation']
                init_yaw = init['yaw']
                target_yaw = target['yaw']
                init_pitch = init['pitch']
                target_pitch = target['pitch']
                
                diff_pitch = abs(init_pitch - target_pitch)
                diff_yaw = abs(init_yaw - target_yaw)
                if diff_yaw > 180:
                    diff_yaw = 360 - diff_yaw
                    
                oracle = diff_pitch + diff_yaw + 1
                
                
                spl = oracle / max(actual, oracle)
            
        spls.append(spl)
        
    
    
    spl = sum(spls) / len(episodes)
    num_perfects /= len(episodes)
    
    return num_perfects, spl

In [30]:
HEADER = ['agent', 'l1', 'stop', 'perfect', 'spl', 'l1', 'stop', 'perfect', 'spl', 'l1', 'stop', 'perfect', 'spl', 'agent', 'env']

def get_results(name, tmp_path, bench_name, agent_names):
    diffs = ['easy', 'medium', 'hard']
    
    assert len(agent_names) == len(diffs)
    
    results = [name]
    agent_times = []
    env_times = []
    for diff in diffs:
        json_path = tmp_path.format(
            bench_name=bench_name,
            diff=diff,
            agent_name=agent_names[diff],
        )
        
        assert os.path.exists(json_path)
        
        with open(json_path, 'r') as f:
            data = json.load(f)
        
        summary = data['summary']
        l1_distance = summary['l1_distance']
        called_stop = summary['called_stop']
        
        episodes = data['episodes_metrics']
        num_perfects, spl = get_num_perfects_with_spl(episodes)
        
        times = data['times']
        agent_times.append(times['agent'])
        env_times.append(times['env'])
        
        result = [l1_distance, called_stop, num_perfects, spl]
        results += result
    
    agent_time = sum(agent_times) / len(agent_times)
    env_time = sum(env_times) / len(env_times)
    results += [agent_time, env_time]
    
    assert len(results) == len(HEADER)
    return results


In [31]:
tmp_path = './benchmarks/{bench_name}/bounded_{diff}/{agent_name}.json'
tmp_bench_name = 'findview_{dataset}_{version}_{category}'
tmp_save_path = './results/{bench_name}.csv'

In [32]:
# sun360 alpha indoor
dataset = 'sun360'
version = 'alpha'
category = 'indoor'
bench_name = tmp_bench_name.format(dataset=dataset, version=version, category=category)

orb_names = {
    'easy': 'fm_orb_inf',
    'medium': 'fm_orb_inf',
    'hard': 'fm_orb_20',
}
sift_names = {
    'easy': 'fm_sift_40',
    'medium': 'fm_sift_50',
    'hard': 'fm_sift_50',
}
ppo_half_names = {
    'easy': 'ppo_inverse2_half',
    'medium': 'ppo_inverse2_half',
    'hard': 'ppo_inverse2_half',
}
ppo_full_names = {
    'easy': 'ppo_inverse2_full',
    'medium': 'ppo_inverse2_full',
    'hard': 'ppo_inverse2_full',
}

In [33]:
results = [HEADER]
results.append(get_results('orb', tmp_path, bench_name, orb_names))
results.append(get_results('sift', tmp_path, bench_name, sift_names))
results.append(get_results('half', tmp_path, bench_name, ppo_half_names))
results.append(get_results('full', tmp_path, bench_name, ppo_full_names))

with open(tmp_save_path.format(bench_name=bench_name), 'w') as f:
    output = csv.writer(f, delimiter=',')
    output.writerows(results)

In [34]:
# sun360 alpha outdoor
dataset = 'sun360'
version = 'alpha'
category = 'outdoor'
bench_name = tmp_bench_name.format(dataset=dataset, version=version, category=category)

orb_names = {
    'easy': 'fm_orb_inf',
    'medium': 'fm_orb_30',
    'hard': 'fm_orb_40',
}
sift_names = {
    'easy': 'fm_sift_inf',
    'medium': 'fm_sift_80',
    'hard': 'fm_sift_40',
}
ppo_indoor_names = {  # use indoor policy
    'easy': 'ppo_inverse2_indoor',
    'medium': 'ppo_inverse2_indoor',
    'hard': 'ppo_inverse2_indoor',
}
ppo_half_names = {
    'easy': 'ppo_inverse2_half',
    'medium': 'ppo_inverse2_half',
    'hard': 'ppo_inverse2_half',
}
ppo_full_names = {
    'easy': 'ppo_inverse2_full',
    'medium': 'ppo_inverse2_full',
    'hard': 'ppo_inverse2_full',
}

In [35]:
results = [HEADER]
results.append(get_results('orb', tmp_path, bench_name, orb_names))
results.append(get_results('sift', tmp_path, bench_name, sift_names))
results.append(get_results('indoor', tmp_path, bench_name, ppo_indoor_names))
results.append(get_results('half', tmp_path, bench_name, ppo_half_names))
results.append(get_results('full', tmp_path, bench_name, ppo_full_names))

with open(tmp_save_path.format(bench_name=bench_name), 'w') as f:
    output = csv.writer(f, delimiter=',')
    output.writerows(results)

In [36]:
# sun360 gamma indoor
dataset = 'sun360'
version = 'gamma'
category = 'indoor'
bench_name = tmp_bench_name.format(dataset=dataset, version=version, category=category)

orb_names = {
    'easy': 'fm_orb_inf',
    'medium': 'fm_orb_inf',
    'hard': 'fm_orb_20',
}
sift_names = {
    'easy': 'fm_sift_40',
    'medium': 'fm_sift_50',
    'hard': 'fm_sift_50',
}
ppo_full_names = {
    'easy': 'ppo_inverse2_full',
    'medium': 'ppo_inverse2_full',
    'hard': 'ppo_inverse2_full',
}

In [37]:
results = [HEADER]
results.append(get_results('orb', tmp_path, bench_name, orb_names))
results.append(get_results('sift', tmp_path, bench_name, sift_names))
results.append(get_results('full', tmp_path, bench_name, ppo_full_names))

with open(tmp_save_path.format(bench_name=bench_name), 'w') as f:
    output = csv.writer(f, delimiter=',')
    output.writerows(results)

In [38]:
# sun360 gamma outdoor
dataset = 'sun360'
version = 'gamma'
category = 'outdoor'
bench_name = tmp_bench_name.format(dataset=dataset, version=version, category=category)

orb_names = {
    'easy': 'fm_orb_inf',
    'medium': 'fm_orb_30',
    'hard': 'fm_orb_40',
}
sift_names = {
    'easy': 'fm_sift_inf',
    'medium': 'fm_sift_80',
    'hard': 'fm_sift_40',
}
ppo_full_names = {
    'easy': 'ppo_inverse2_full',
    'medium': 'ppo_inverse2_full',
    'hard': 'ppo_inverse2_full',
}

In [39]:
results = [HEADER]
results.append(get_results('orb', tmp_path, bench_name, orb_names))
results.append(get_results('sift', tmp_path, bench_name, sift_names))
results.append(get_results('full', tmp_path, bench_name, ppo_full_names))

with open(tmp_save_path.format(bench_name=bench_name), 'w') as f:
    output = csv.writer(f, delimiter=',')
    output.writerows(results)

In [40]:
tmp_bench_name = 'findview_{dataset}_{version}'

In [41]:
# wacv alpha indoor
dataset = 'wacv360indoor'
version = 'alpha'
bench_name = tmp_bench_name.format(dataset=dataset, version=version)

orb_names = {
    'easy': 'fm_orb_30',
    'medium': 'fm_orb_30',
    'hard': 'fm_orb_20',
}
sift_names = {
    'easy': 'fm_sift_80',
    'medium': 'fm_sift_50',
    'hard': 'fm_sift_50',
}
ppo_sun360_names = {
    'easy': 'ppo_inverse2_sun360',
    'medium': 'ppo_inverse2_sun360',
    'hard': 'ppo_inverse2_sun360',
}
ppo_finetuned_names = {
    'easy': 'ppo_inverse2_finetuned',
    'medium': 'ppo_inverse2_finetuned',
    'hard': 'ppo_inverse2_finetuned',
}

In [42]:
results = [HEADER]
results.append(get_results('orb', tmp_path, bench_name, orb_names))
results.append(get_results('sift', tmp_path, bench_name, sift_names))
results.append(get_results('sun360', tmp_path, bench_name, ppo_sun360_names))
results.append(get_results('finetuned', tmp_path, bench_name, ppo_finetuned_names))

with open(tmp_save_path.format(bench_name=bench_name), 'w') as f:
    output = csv.writer(f, delimiter=',')
    output.writerows(results)

In [43]:
# wacv gamma indoor
dataset = 'wacv360indoor'
version = 'gamma'
bench_name = tmp_bench_name.format(dataset=dataset, version=version)

orb_names = {
    'easy': 'fm_orb_30',
    'medium': 'fm_orb_30',
    'hard': 'fm_orb_20',
}
sift_names = {
    'easy': 'fm_sift_80',
    'medium': 'fm_sift_50',
    'hard': 'fm_sift_50',
}
ppo_sun360_names = {
    'easy': 'ppo_inverse2_sun360',
    'medium': 'ppo_inverse2_sun360',
    'hard': 'ppo_inverse2_sun360',
}
ppo_finetuned_names = {
    'easy': 'ppo_inverse2_finetuned',
    'medium': 'ppo_inverse2_finetuned',
    'hard': 'ppo_inverse2_finetuned',
}

In [44]:
results = [HEADER]
results.append(get_results('orb', tmp_path, bench_name, orb_names))
results.append(get_results('sift', tmp_path, bench_name, sift_names))
results.append(get_results('sun360', tmp_path, bench_name, ppo_sun360_names))
results.append(get_results('finetuned', tmp_path, bench_name, ppo_finetuned_names))

with open(tmp_save_path.format(bench_name=bench_name), 'w') as f:
    output = csv.writer(f, delimiter=',')
    output.writerows(results)